# $\mathcal{l}_2$ penalty method algorithm

In [1]:
import matplotlib.pyplot as plt
import random
import numpy as np
import time
from copy import deepcopy


In [39]:
def alpha(k): return 2**k
def xplus(x): return max(x,0)
def obj_func(x): return x[0]**2 + x[1]**2 + x[0] - x[1]
def G(x): return np.array([1 -x[0], -x[1] ])
def P(x,k): return obj_func(x) + (alpha(k)/2)*xplus(G(x)[0])**2 + (alpha(k)/2)*xplus(G(x)[1])**2
def P_grad(x,k):
    g1 = 2*x[0] + 1 - alpha(k)*xplus(1-x[0])
    g2 = 2*x[1] - 1 - alpha(k)*xplus(-x[1])
    return np.array( [ g1 , g2 ] )
def Norm(x): return np.linalg.norm(x)
def stepsize(x1, x0,k): # Barzilai-Borwein step-size (quasi-newton origin)
    nom = np.dot(x1 - x0, P_grad(x1,k) - P_grad(x0,k))
    denom = np.linalg.norm( P_grad(x1,k) - P_grad(x0,k) )**2
    return nom/denom
# Note: Direction is just steepest descent
def GD(x, lam ,k):
    return x - lam*P_grad(x,k)

In [ ]:
def Descent(x,k, EPSILON):
    x0 = np.array(x)
    k = 1
    x_list = [x0]
    lam = random.random() + 1e-12
    x1 = GD(x0, lam,k)
    x_list.append(x1)
    
    
    

In [63]:
x0 = np.array([5,5])
EPSILON = 1e-6
k = 1
# x0 = np.array([random.randrange(1,20), random.randrange(0,20)])
print("x:", x0)
print("Gradient", P_grad(x0,k))
lam = random.random() + 1e-12
print('stepsize:',lam)
x1 = GD(x0, lam,k)
print("x:", x1)
print("Gradient", P_grad(x1,k))
N = Norm(x1 - x0)
print("Squared Norm:",N)
while N > EPSILON:
    lam = stepsize(x1,x0, k)
    print("stepsize:", lam)
    x0 = x1
    x1 = GD(x0, lam, k)
    print("x:", x1)
    print("Gradient", P_grad(x1,k))
    k += 1
    print("k=", k)
    N = Norm(x1 - x0)
    print("Squared Norm:",N)

x: [5 5]
Gradient [11  9]
stepsize: 0.8208050707183439
x: [-4.02885578 -2.38724564]
Gradient [-17.11542311 -10.54898255]
Squared Norm: 11.665831935683928
stepsize: 0.33962966154213653
x: [1.78404958 1.19550174]
Gradient [4.56809916 1.39100347]
k= 2
Squared Norm: 6.82831951775667
stepsize: 0.1899109468744791
x: [0.91651754 0.93133495]
Gradient [2.49910526 0.8626699 ]
k= 3
Squared Norm: 0.9068604778979921
stepsize: 0.36744024445033624
x: [0.12094495 0.61435531]
Gradient [-5.79055048  0.22871062]
k= 4
Squared Norm: 0.8563946746811998
stepsize: 0.056424886740392255
x: [0.84448076 0.60145034]
Gradient [0.20065367 0.20290068]
k= 5
Squared Norm: 0.7236508840587734
stepsize: 0.029412282714217683
x: [0.91176589 0.59548257]
Gradient [4.02908139e-05 1.90965135e-01]
k= 6
Squared Norm: 0.0675492654103332
stepsize: 0.015155017515508603
x: [0.95455534 0.59258849]
Gradient [0.00065266 0.18517698]
k= 7
Squared Norm: 0.04288721178836533
stepsize: 0.007692840728916017
x: [0.97692463 0.59116395]
Gradient 

In [65]:
Norm([5.12295895e-09, 1.79508469e-01])**2

0.03222329044272399

# OLD CODE

## Failed $\log$ barrier penalty method algorithm

In [ ]:
# testing
def obj_func(x): return x[0]**2 + x[1]**2 + x[0] - x[1]

In [ ]:
def Gs(x):
    g1, g2 = 1 - x[0] , -x[1]
    return np.array([g1,g2])

In [ ]:
pre_lim_x_test = np.array([2,1.5])
print(obj_func(pre_lim_x_test))
Gs(pre_lim_x_test)

In [ ]:
def P(x,k):
    return obj_func(x) - (1/k)*np.log(-Gs(x))

In [ ]:
(1/k)*np.log(Gs(x1))

In [ ]:
def P_grad(x,k):
    p1 = 2*x[0] + 1 + 1/(k*(1-x[0]))
    p2 = 2*x[1] -1 + 1/(k*x[1])
    return np.array([p1,p2])

In [ ]:
P_grad(pre_lim_x_test,1)

In [ ]:
def Norm(x): return np.linalg.norm(x)

def stepsize(x1, x0,k):
    nom = np.dot(x1 - x0, P_grad(x1,k) - P_grad(x0,k))
    denom = np.linalg.norm( P_grad(x1,k) - P_grad(x0,k) )**2
    return nom/denom

def GD(x, lam ,k):
    return x - lam*P_grad(x,k)

In [ ]:
x = pre_lim_x_test
EPSILON = 1e-3
k = 1
# x0 = np.array([random.randrange(1,20), random.randrange(0,20)])
x0 = x
print("x:", x0)
print("Gradient", P_grad(x0,k))
lam = random.random() + 1e-12


In [ ]:
(1/k)*np.log(Gs(x0))

In [ ]:
Gs(x0)

In [ ]:
x1 = GD(x0, lam,k)
print("x:", x1)
print("Gradient", P_grad(x1,k))
N = Norm(P_grad(x,k))
print("Squared Norm:",N)

In [ ]:
Norm(P_grad([1.0000000001,0.0000000001],k))

In [ ]:
while N > EPSILON:
    lam = stepsize(x1,x0, k)
    print("stepsize:", lam)
    x0 = x1
    x1 = GD(x0, lam, k)
    print("x:", x1)
    print("Gradient", P_grad(x1,k))
    k += 1
    N = Norm(P_grad(x,k))
    print("Squared Norm:",N)
            
           

In [ ]:
k

In [ ]:
FIELDSIZE = 100
NUM_X = 100

X = []

for i in range(FIELDSIZE):
    x_i = np.array([random.randrange(-FIELDSIZE, FIELDSIZE), random.randrange(-FIELDSIZE, FIELDSIZE)])
    X.append(x_i)

In [ ]:
S = np.array([random.randrange(-FIELDSIZE, FIELDSIZE), random.randrange(-FIELDSIZE, FIELDSIZE)])
S_data = [S]

def norm(s, x):
    return (s[0] - x[0])**2 + (s[1] - x[1])**2

norms = []
for x in X:
    norms.append(norm(S,x))

alpha_index = np.argmax(norms)
alpha = norms[alpha_index]

In [ ]:
alpha

In [ ]:
sum(norms)

In [ ]:
lambdas = [0]*len(X)

def obj(s):
    obj_norms = []
    for x in X:
        obj_norms.append(norm(S,x))
    return sum(obj_norms) + alpha

In [ ]:
def conditions(s):
    g = [0]*len(X)
    for i in range(len(X)):
        g[i] = norm(s, X[i]) - alpha
    return g

In [ ]:
def lagrange(s):
    G = conditions(s)
    lambda_gs = [0]*len(X)
    for i in range(len(X)):
        lambda_gs[i] = lambdas[i]*G[i]
    return obj(s) + sum(lambda_gs)

In [ ]:
lagrange(S)

In [ ]:
obj(S)

In [ ]:
def grad_lagrange(s):
    gradL = [0, 0]
    for i in range(len(X)):
        gradL[0] += 2*(1+lambdas[i])*(s[0]-X[i][0])
        gradL[1] += 2*(1+lambdas[i])*(s[1]-X[i][1])
    return np.array(gradL)

In [ ]:
G = conditions(S)
G[alpha_index]

In [ ]:
def grad_gl(s):
    gradgL = [0,0]
    for i in range(len(X)):
        gradgL[0] += 2*(s[0] - X[i][0])
        gradgL[1] += 2*(s[1] - X[i][1])
    return np.array(gradgL)

In [ ]:
def Real_norm(s): return np.linalg.norm(s)

In [ ]:
def step(lambdaa):
    return lambdaa - STEPSIZE*grad_gl(S)

In [ ]:
def Backtrack(l1, l2):
    STEPSIZE = np.dot(l2 - l1, 2*(S - ))

In [ ]:
teste = []
L1 = random.randrange(1e6)
L2 = L1 - 5
while Real_norm(grad_lagrange(S)) > 10:
    STEPSIZE = Backtrack(L1, L2)
    lambdas[alpha_index]  = step(lambdas[alpha_index])
    teste.append(lambdas[alpha_index])

In [ ]:
plt.plot(range(len(teste)), teste)
plt.show

In [ ]:
teste[-1]

In [ ]:
teste[0]

In [ ]:
Real_norm(S - np.array(grad_lagrange(S)))

In [ ]:
grad_lagrange(S)

In [ ]:
grad_lagrange(S - 10 * np.array(grad_lagrange(S)))

In [ ]:
STEPSIZE

In [ ]:
random.randrange(1e6)